In [86]:
import pandas as pd
import re

# Load the CSV file
df = pd.read_csv('df_clean.csv')

# Display the column names
print(df.columns.tolist())
print(len(df))

# Display the first 5 rows in a nice table
display(df['schedule'].head())

['url', 'title', 'volume', 'education', 'content', 'learning_outcome', 'literature', 'recommended_prereq', 'teaching_methods', 'workload', 'feedback_form', 'signup', 'exam_html', 'language', 'course_code', 'ects', 'level', 'duration', 'placement', 'schedule', 'capacity', 'study_board', 'department', 'faculty', 'course_coordinators', 'last_modified', 'term']
792


0    A
1    C
2    C
3    A
4    A
Name: schedule, dtype: object

In [87]:
#def ()
print(len("Volume 2025/2026_Block 1"))

24


In [88]:
def clean_term(val):
    if pd.isna(val):
        return val
    val_str = str(val) 
    first_year = str(val_str)[9:11]
    second_year = str(val_str)[14:16]
    if len(val_str)==24:
        block = "B"+str(val_str)[-1]
    elif "Summer" in val_str and not "Blok" in val_str:
        block = "S"
    else:
        block = ""
    return "s"+first_year+"/"+second_year+block

for orig, cleaned in zip(df['term'], df['term'].apply(clean_term)):
    print(f"Original: {orig} -> Cleaned: {cleaned}")
df['term'] = df['term'].apply(clean_term)

Original: Volume 2024/2025_Block 2 -> Cleaned: s24/25B2
Original: Volume 2024/2025_Block 2 -> Cleaned: s24/25B2
Original: Volume 2025/2026_Block 2 -> Cleaned: s25/26B2
Original: Volume 2025/2026_Block 2 -> Cleaned: s25/26B2
Original: Volume 2025/2026_Block 1 -> Cleaned: s25/26B1
Original: Volume 2024/2025_Block 1 -> Cleaned: s24/25B1
Original: Volume 2025/2026_Block 2 -> Cleaned: s25/26B2
Original: Volume 2024/2025_Block 2 -> Cleaned: s24/25B2
Original: Volume 2024/2025_Block 2 -> Cleaned: s24/25B2
Original: Volume 2025/2026_Block 2 -> Cleaned: s25/26B2
Original: Volume 2025/2026_Block 1 -> Cleaned: s25/26B1
Original: Volume 2024/2025_Block 1 -> Cleaned: s24/25B1
Original: Volume 2024/2025_Block 1 -> Cleaned: s24/25B1
Original: Volume 2025/2026_Block 1 -> Cleaned: s25/26B1
Original: Volume 2024/2025_Block 3 -> Cleaned: s24/25B3
Original: Volume 2025/2026_Block 1 -> Cleaned: s25/26B1
Original: Volume 2024/2025_Block 1 -> Cleaned: s24/25B1
Original: Volume 2025/2026_Block 3 -> Cleaned: s

In [ ]:
def split_coordinator(row):
    match = re.match(r"\['(.*)\s\((.*)\)'\]", str(row))
    if match:
        return pd.Series({'course_coordinator_name': match.group(1), 'course_coordinator_id': match.group(2)})
    else:
        return pd.Series({'course_coordinator_name': None, 'course_coordinator_id': None})

split_df = df['course_coordinators'].apply(split_coordinator)
df['course_coordinator_name'] = split_df['course_coordinator_name']
df['course_coordinator_id'] = split_df['course_coordinator_id']

def single_course_coordinator(val):
    if pd.isna(val):
        return val
    match = re.match(r"(.*)\s\(.*", str(val))
    if match:
        return match.group(1)
    else:
        return val
    
df['course_coordinator_name'] = df['course_coordinator_name'].apply(single_course_coordinator)

def clean_schedule(val):
    if pd.isna(val):
        return val
    val_str = str(val)
    if len(val_str) == 2:
        return val_str[0]
    elif len(val_str) > 30:
        if "Summer" in val_str and not "Blok" in val_str:
            return "Summer"
        elif "Sommer" in val_str and not "Blok" in val_str:
            return "Summer"
        else:
            return "Other"
    else:
        return val_str

df['schedule'] = df['schedule'].apply(clean_schedule)

# Now df has two new columns
display(df[['course_coordinator_name', 'course_coordinator_id']].head())
df.to_csv('coordinator_split.csv', index=False)

,course_coordinator_name,course_coordinator_id
0,Panagiotis Karras,4-7566706645696e33707a336970
1,Amir Yehudayoff,4-6975816d486c7136737d366c73
2,Jacob Holm,4-6d646b7243676c316e7831676e
3,Panagiotis Karras,4-72636d6342666b306d7730666d
4,Troels Henriksen,5-71847871835074793e7b853e747b


In [90]:
print(single_course_coordinator("John Doe (12345)"))

John Doe (12345)


In [91]:
def unique_code_kuc_all_columns(df):
    """
    Returns a new DataFrame with all columns, keeping only the first occurrence of each unique code_kuc.
    """
    return df.drop_duplicates(subset=['course_code']).reset_index(drop=True)

# Example usage:
unique_df = unique_code_kuc_all_columns(df)
display(unique_df['course_coordinator_name'])
print(type(unique_df['course_coordinator_name'][0]))

0                                     Panagiotis Karras
1                                       Amir Yehudayoff
2                                      Troels Henriksen
3                                         Stefan Sommer
4                               Joanna Emilia Bergström
5                                         Jens Petersen
6                                        Amartya Sanyal
7                                       Desmond Elliott
8                                        Fritz Henglein
9                                  Abraham George Smith
10                                                 None
11                                  Srikanth Srinivasan
12                                        Rasmus Heller
13                                        Yongluan Zhou
14                                        Rasmus Heller
15                                        Rasmus Heller
16                                        Rasmus Heller
17                                     Alexander

<class 'str'>


In [92]:
unique_df.to_csv('unique_courses.csv', index=False)

In [93]:
print(unique_df['content'].str.len().max())

3231


In [94]:
import pandas as pd

pd.set_option('display.max_rows', None)  # Show all rows
print(unique_df['schedule'])

0                             A
1                             C
2                             A
3                             A
4                             C
5                             B
6                             C
7                             A
8                             B
9                         Other
10                          NaN
11                            A
12    Meetings with supervisor.
13                            C
14    Meetings with supervisor.
15    Meetings with supervisor.
16    Meetings with supervisor.
17                            C
18                            C
19                            C
20                            A
21                            C
22                            B
23                            C
24                            A
25                     B1 And D
26                            C
27                            A
28                            C
29                            B
30                            B
31      